## Импорт библиотек

In [1]:
import pandas as pd
import sqlite3

## Соединение с базой данных и схема таблицы

In [2]:
connection = sqlite3.connect('../data/checking-logs.sqlite')
query = "PRAGMA table_info(checker);"
scheme = pd.read_sql(query, connection)
scheme

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


## A/B тестирование

In [3]:
test_query = "SELECT 'before' AS time, \
    ROUND(AVG((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24), 2) AS avg_diff \
        FROM test t JOIN deadlines d ON t.labname = d.labs WHERE t.labname LIKE 'laba%' AND t.first_commit_ts < t.first_view_ts \
            UNION ALL \
                SELECT 'after' AS time, ROUND(AVG((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24), 2) AS avg_diff \
                    FROM test t JOIN deadlines d ON t.labname = d.labs WHERE t.labname LIKE 'laba%' AND t.first_commit_ts > t.first_view_ts;"

test_results = pd.read_sql(test_query, connection)
test_results

,time,avg_diff
0,before,-61.16
1,after,-103.95


In [4]:
control_query = "SELECT 'before' AS time, \
    ROUND(AVG((JULIANDAY(c.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24), 2) AS avg_diff \
        FROM control c JOIN deadlines d ON c.labname = d.labs WHERE c.labname LIKE 'laba%' AND c.first_commit_ts <= c.first_view_ts \
            UNION ALL \
                SELECT 'after' AS time, ROUND(AVG((JULIANDAY(c.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24), 2) AS avg_diff \
                    FROM control c JOIN deadlines d ON c.labname = d.labs WHERE c.labname LIKE 'laba%' AND c.first_commit_ts > c.first_view_ts;"
control_results = pd.read_sql(control_query, connection)
control_results

,time,avg_diff
0,before,-99.90
1,after,-113.23


## Закрытие соединения с базой данных

In [5]:
connection.close()

## Гипотеза оказалась верной и страница действительно влияет на поведение студентов?

### Нет